In [1]:
# f_ufl = ufl.Constant(domain_mesh, (PETSc.ScalarType(0), PETSc.ScalarType(0)))
# f_fem = fem.Constant(domain_mesh, (PETSc.ScalarType(0), PETSc.ScalarType(0)))

# g_ufl = ufl.Constant(domain_mesh, PETSc.ScalarType(0))
# g_fem = fem.Constant(domain_mesh, PETSc.ScalarType(0))

# a_block = fem.form(
#     [
#         [(grad(u)|dot|grad(v)) * dx, (p|dot|div(v)) * dx],
#         [(div(u)|dot|q) * dx, None]
#         ]
#     )
# a_p11 = fem.form((p|dot|q) * dx)
# a_p = [[a_block[0][0], None], [None, a_p11]]
# L_block = fem.form([(f_fem|dot|v) * dx, (g_fem|dot|q) * dx])

In [2]:
import os
import shutil
import sys
import time

import numpy as np
import copy

from dolfinx import fem, mesh, io, nls
from dolfinx import plot as dofplot
from ufl import dx, ds, grad, inner, exp, sin, diff, div
import ufl
from mpi4py import MPI
from petsc4py import PETSc

from ipywidgets import IntProgress
from IPython.display import display
import matplotlib.pyplot as plt
import pyvista as pv

In [3]:
class Infix:

    def __init__(self, function):
        self.function = function

    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))

    def __or__(self, other):
        return self.function(other)

    def __call__(self, value1, value2):
        return self.function(value1, value2)

In [4]:
dot = Infix(inner)

In [5]:
def draw_plot(space, points, func):
    c_topology, c_cell_types, c_geometry = dofplot.create_vtk_mesh(space)
    check = np.vstack((np.arange(len(func)), func)).T[points][:, 1]
    c_data = np.column_stack((c_geometry[:, 0:2], check))
    x_data = c_data[:, 0]
    y_data = c_data[:, 1]
    z_data = c_data[:, 2]
    plt.tripcolor(x_data, y_data, z_data, shading='gouraud')

In [6]:
domain_mesh = mesh.create_rectangle(
    points=((0, 0), (1, 1)),
    n=(32, 32),
    cell_type=mesh.CellType.triangle,
    ghost_mode=mesh.GhostMode.none,
    comm=MPI.COMM_WORLD,
    )
u_elem = ufl.VectorElement("Lagrange", domain_mesh.ufl_cell(), 2)
p_elem = ufl.FiniteElement("Lagrange", domain_mesh.ufl_cell(), 1)
V, P = fem.FunctionSpace(domain_mesh,u_elem),fem.FunctionSpace(domain_mesh,p_elem)
GEN = fem.FunctionSpace(domain_mesh, u_elem * p_elem)
R_V, _ = GEN.sub(0).collapse()

### Standart Solver

In [7]:
bc_static_x = fem.Function(V)
bc_static_marker = mesh.locate_entities_boundary(
    domain_mesh,
    dim=1,
    marker=lambda x: np.isclose(x[0], 0)|np.isclose(x[0], 1)
    |np.isclose(x[1], 0)
    )
bc_static_dofs = fem.locate_dofs_topological(
    V=(GEN.sub(0), V),
    entity_dim=1,
    entities=bc_static_marker,
    )
bc_static = fem.dirichletbc(
    bc_static_x,
    bc_static_dofs,
    GEN.sub(0),     # V=GEN.sub(0),
     # value=bc_static_x,
     # dofs=bc_static_dofs,
    )


def bc_dynamic_x_expression(x):
    return np.stack((np.ones(x.shape[1]), np.zeros(x.shape[1])))


bc_dynamic_x = fem.Function(R_V)
bc_dynamic_x.interpolate(bc_dynamic_x_expression)
bc_dynamic_marker = mesh.locate_entities_boundary(
    domain_mesh,
    dim=1,
    marker=lambda x: np.isclose(x[1], 1),
    )
bc_dynamic_dofs = fem.locate_dofs_topological(
    (GEN.sub(0), V),
    1,
    bc_dynamic_marker,
    )
bc_dynamic = fem.dirichletbc(
    bc_dynamic_x,
    bc_dynamic_dofs,
    GEN.sub(0),
    )

bc_init_x = fem.Function(P)
bc_init_x.x.set(0.0)
bc_init_dofs = fem.locate_dofs_geometrical(
    V=(GEN.sub(1), P),
    marker=lambda x: np.isclose(x.T, [0, 0, 0]).all(axis=1),
    )
bc_init = fem.dirichletbc(
    V=GEN.sub(1),
    dofs=bc_init_dofs,
    value=bc_init_x,
    )
bcs = [bc_static, bc_dynamic, bc_init]

In [8]:
u, p = ufl.TrialFunctions(GEN)
v, q = ufl.TestFunctions(GEN)
f = fem.Function(R_V)

a = (grad(u)|dot|grad(v)) * dx + (p|dot|div(v)) * dx + (div(u)|dot|q) * dx
a = fem.form(a)
left = (f|dot|v) * dx
left = fem.form(left)


In [9]:
a_matrix = fem.petsc.assemble_matrix(a, bcs=bcs)
a_matrix.assemble()
b = fem.petsc.assemble_vector(left)
fem.petsc.apply_lifting(b, [a], bcs=[bcs])
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

# Set Dirichlet boundary condition values in the RHS
fem.petsc.set_bc(b, bcs)

# Create and configure solver
ksp = PETSc.KSP().create(domain_mesh.comm)
ksp.setOperators(a_matrix)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("superlu_dist")

In [10]:
func = fem.Function(GEN)
ksp.solve(b, func.vector)

In [11]:
result_u = func.sub(0).collapse()
result_p = func.sub(1).collapse()

print(
    f'Norm of velocity: {result_u.x.norm()}  Norm of pressure:{result_p.x.norm()}'
    )
with io.XDMFFile(MPI.COMM_WORLD, 'Stokes_file/velocity.xdmf', 'w') as file:
    result_u.x.scatter_forward()
    file.write_mesh(domain_mesh)
    file.write_function(result_u)
with io.XDMFFile(MPI.COMM_WORLD, 'Stokes_file/pressure.xdmf', 'w') as file:
    result_p.x.scatter_forward()
    file.write_mesh(domain_mesh)
    file.write_function(result_p)


Norm of velocity: 17.47837015708121  Norm of pressure:311.5914181997778
